- berapa rata2 barang yang dipesan pada sekali pemesanan?
- jam, hari, bulan yang paling sering order?
- barang apa yang paling dibutuhkan setiap segmen
- segmen yg paling profit
- rata2 profit setiap pemesanan pada masing2 segmen
- apakah barang2 diskon tinggi lebih sering dipesan

This data is contains of sales record included the profit or loss from a retail company (i named it PT. Elips). PT.Elips is a global retail company who sells Office Supplies, Furniture and Technology stuff.
The buyers are coming from different segment such as Corporate, Home Office until individual consumer. This company also could provide the best option for your delivery which suitable with your needs.

Note : Detail explanatory will be given at the end of content

In [187]:
import numpy as np
import pandas as pd

- Row ID              : ID data
- Order ID            : ID pemesanan
- Order Date          : Tanggal pemesanan
- Ship Date           : Tanggal pengiriman
- Ship Mode           : Mode pengiriman
- Customer ID         : ID si pemesan
- Segment             : Segmentasi si pemesan
- Product ID          : ID barang
- Category            : Kategori barang
- Sub-Category        : Sub-kategori barang
- Product Name        : Nama produk
- Sales               : Penjualan
- Quantity            : Kuantitas barang yang dipesan
- Discount            : Diskon barang
- Profit              : Profit yang dihasilkan

## Import Dataset

In [5]:
retail = pd.read_csv('retail.csv', encoding='ISO-8859-1')
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Row ID        9994 non-null   int64  
 1   Order ID      9994 non-null   object 
 2   Order Date    9994 non-null   object 
 3   Ship Date     9994 non-null   object 
 4   Ship Mode     9994 non-null   object 
 5   Customer ID   9994 non-null   object 
 6   Segment       9994 non-null   object 
 7   Product ID    9994 non-null   object 
 8   Category      9994 non-null   object 
 9   Sub-Category  9994 non-null   object 
 10  Product Name  9994 non-null   object 
 11  Sales         9994 non-null   float64
 12  Quantity      9994 non-null   int64  
 13  Discount      9994 non-null   float64
 14  Profit        9994 non-null   float64
dtypes: float64(3), int64(2), object(10)
memory usage: 1.1+ MB


- Dataset terdiri dari 9994 record data dan 15 feature columns
- tidak ada missing value
- 5 tipe data numerik, yang salah satunya id
- cek kolom yang bisa diubah ke categorical dan datetime

In [61]:
retail['Order Date'] = pd.to_datetime(retail['Order Date'], format='%m/%d/%y')
retail['Ship Date'] = pd.to_datetime(retail['Ship Date'], format='%m/%d/%y')

categorical_col = ['Ship Mode', 'Segment', 'Category', 'Sub-Category']
retail[categorical_col] = retail[categorical_col].astype('category')
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Row ID        9994 non-null   int64         
 1   Order ID      9994 non-null   object        
 2   Order Date    9994 non-null   datetime64[ns]
 3   Ship Date     9994 non-null   datetime64[ns]
 4   Ship Mode     9994 non-null   category      
 5   Customer ID   9994 non-null   object        
 6   Segment       9994 non-null   category      
 7   Product ID    9994 non-null   object        
 8   Category      9994 non-null   category      
 9   Sub-Category  9994 non-null   category      
 10  Product Name  9994 non-null   object        
 11  Sales         9994 non-null   float64       
 12  Quantity      9994 non-null   int64         
 13  Discount      9994 non-null   float64       
 14  Profit        9994 non-null   float64       
dtypes: category(4), datetime64[ns](2), flo

In [252]:
retail['Order Date'].min()

Timestamp('2014-01-03 00:00:00')

In [253]:
retail['Order Date'].max()

Timestamp('2017-12-30 00:00:00')

Dataset berisi informasi order mulai dari tanggal **03 Januari 2014 - 30 Desember 2017**

## EDA

### Berapa rata2 barang yang dipesan pada sekali pemesanan?

In [64]:
retail.Quantity.mean()

3.789573744246548

In [68]:
retail['Order ID'].nunique()

5009

In [75]:
order_product_num = retail.groupby('Order ID')['Product ID'].count()
order_product_num

Order ID
CA-2014-100006    1
CA-2014-100090    2
CA-2014-100293    1
CA-2014-100328    1
CA-2014-100363    2
                 ..
US-2017-168802    1
US-2017-169320    2
US-2017-169488    2
US-2017-169502    2
US-2017-169551    6
Name: Product ID, Length: 5009, dtype: int64

In [79]:
order_product_num.mean()

1.9952086244759433

>Dari 9.994 data terdapat 5.009 kali peng-orderan yang mana untuk setiap order yang dilakukan rata-rata mereka memesan hampir **2 barang** yang berbeda sekaligus.

### Hari dan bulan yang paling sering order?

In [131]:
retail['Day Order'] = retail['Order Date'].dt.day_name().astype('category')
retail['Month Order'] = retail['Order Date'].dt.month_name().astype('category')

In [132]:
unique_order = retail[['Order ID', 'Day Order', 'Month Order']].drop_duplicates()
num_order = unique_order.shape[0]
unique_order.head()

,Order ID,Day Order,Month Order
0,CA-2016-152156,Tuesday,November
2,CA-2016-138688,Sunday,June
3,US-2015-108966,Sunday,October
5,CA-2014-115812,Monday,June
12,CA-2017-114412,Saturday,April


In [133]:
unique_order.groupby('Day Order')['Order ID'].count().sort_values(ascending=False) / num_order * 100

Day Order
Monday       18.366940
Friday       18.287083
Sunday       16.969455
Saturday     16.709922
Thursday     14.893192
Tuesday      11.139948
Wednesday     3.633460
Name: Order ID, dtype: float64

>Jika memperhatikan order yang dilakukan per harinya maka bisa dilihat jika order paling sering dilakukan pada hari Senin dan Jumat. Dan yang paling mengejutkan ialah order yang paling sedikit yaitu pada **hari Rabu** yang hanya 3.63%, yang mana proporsinya sangat timpang jika dibandingkan dengan hari lainnya.

### Barang dengan jenis sub-kategori apa yang paling dibutuhkan setiap segmen?

In [165]:
segment_order = retail.groupby(['Segment', 'Sub-Category'])['Row ID'].count()
consumer_order = segment_order.unstack().loc['Consumer'].sort_values(ascending=False)
consumer_order / sum(consumer_order) * 100

Sub-Category
Binders        15.026007
Paper          13.350029
Furnishings     9.516471
Phones          8.688114
Storage         8.668850
Art             8.245039
Accessories     7.859757
Chairs          6.337893
Appliances      4.700443
Labels          3.640917
Tables          3.062994
Bookcases       2.523599
Envelopes       2.485070
Fasteners       2.196109
Supplies        1.926411
Machines        1.098054
Copiers         0.674244
Name: Consumer, dtype: float64

In [166]:
corporate_order = segment_order.unstack().loc['Corporate'].sort_values(ascending=False)
corporate_order / sum(corporate_order) * 100

Sub-Category
Binders        15.662252
Paper          13.509934
Furnishings     9.536424
Phones          9.006623
Storage         8.609272
Accessories     7.516556
Art             7.218543
Chairs          6.225166
Appliances      5.132450
Tables          3.509934
Labels          3.377483
Envelopes       2.781457
Bookcases       2.119205
Supplies        2.019868
Fasteners       1.953642
Machines        1.158940
Copiers         0.662252
Name: Corporate, dtype: float64

In [167]:
home_office_order = segment_order.unstack().loc['Home Office'].sort_values(ascending=False)
home_office_order / sum(home_office_order) * 100

Sub-Category
Binders        15.143017
Paper          15.086932
Furnishings     9.814919
Phones          9.310151
Art             8.412787
Accessories     7.851935
Storage         7.627594
Chairs          5.608525
Labels          4.094223
Appliances      3.757712
Tables          3.028603
Fasteners       2.467751
Envelopes       2.299495
Bookcases       1.850813
Supplies        1.626472
Machines        1.289961
Copiers         0.729108
Name: Home Office, dtype: float64

>Berdasarkan order yang dilakukan oleh setiap segmen, sub-kategori produk yang di-order hampir sama saja. Dengan kata lain semua segmen memiliki kecenderungan untuk **memesan sub-kategori yang sama**. Dari keseluruhan order yang dilakukan sekitar seperempatnya berasal dari **sub-kategori binders dan paper**.

### Segmen apa yang paling besar menghasilkan profit?

In [172]:
retail.groupby('Segment')['Profit'].sum().sort_values(ascending=False)

Segment
Consumer       134119.2092
Corporate       91979.1340
Home Office     60298.6785
Name: Profit, dtype: float64

### Segmen apa yang menghasilkan rata2 profit tertinggi?

In [173]:
retail.groupby('Segment')['Profit'].mean().sort_values(ascending=False)

Segment
Home Office    33.818664
Corporate      30.456667
Consumer       25.836873
Name: Profit, dtype: float64

In [174]:
retail['Segment'].value_counts()

Consumer       5191
Corporate      3020
Home Office    1783
Name: Segment, dtype: int64

>Profit terbanyak dihasilkan oleh segmen Consumer. Namun jika kita menghitung rata-rata yang dihasilkan setiap segmennya, maka segmen Home Office lah yang menghasilkan profit terbesar walaupun frekuensi order yang dilakukan oleh segmen ini adalah yang terendah. Hal ini menunjukkan bahwa order akan menghasilkan profit tertinggi jika yang melakukan order berada dalam segmen Home Office.

### Apakah barang2 diskon tinggi lebih menghasilkan profit yang lebih tinggi pula?

In [180]:
retail['Discount'].unique()

array([0.  , 0.45, 0.2 , 0.8 , 0.3 , 0.5 , 0.7 , 0.6 , 0.32, 0.1 , 0.4 ,
       0.15])

In [210]:
def convert_discount(x):
    if x == 0:
        return 'No Discount'
    elif x>0 and x<.3:
        return 'Discount < 30%'
    elif x>=.3 and x<=.6:
        return 'Discount 30% - 60%'
    elif x>.6:
        return 'Discount > 60%'
    
discount = retail['Discount'].map(convert_discount).values
discount_df = pd.DataFrame({'Discount': discount, 'Profit': retail.Profit})
discount_df.groupby('Discount')['Profit'].sum().sort_values(ascending=False)

Discount
No Discount           320987.6032
Discount < 30%        100785.4745
Discount 30% - 60%    -64761.6599
Discount > 60%        -70614.3961
Name: Profit, dtype: float64

>Diskon dengan besaran 30% keatas justru cenderung memberikan kerugian. Sebaliknya barang-barang dengan diskon dibawah 30% dan bahkan barang-barang tanpa diskon meenghasilkan profit yang besar. Sepertinya besarnya **diskon hanya dijadikan sebagai strategi marketing untuk memikat pelanggan**, bukan untuk mencari profit.

### Segmen apa yang paling mendominasi untuk setiap ship mode nya?

In [232]:
shipmode_order = retail[['Row ID', 'Ship Mode', 'Segment']].copy()
shipmode_order = shipmode_order.groupby(['Ship Mode', 'Segment'])['Row ID'].count().unstack()

In [233]:
shipmode_order.loc['Standard Class'] / shipmode_order.loc['Standard Class'].sum() * 100

Segment
Consumer       51.692359
Corporate      30.361930
Home Office    17.945710
Name: Standard Class, dtype: float64

In [234]:
shipmode_order.loc['Second Class'] / shipmode_order.loc['Second Class'].sum() * 100

Segment
Consumer       52.442159
Corporate      31.311054
Home Office    16.246787
Name: Second Class, dtype: float64

In [235]:
shipmode_order.loc['First Class'] / shipmode_order.loc['First Class'].sum() * 100

Segment
Consumer       50.00000
Corporate      31.53446
Home Office    18.46554
Name: First Class, dtype: float64

In [236]:
shipmode_order.loc['Same Day'] / shipmode_order.loc['Same Day'].sum() * 100

Segment
Consumer       58.379374
Corporate      20.994475
Home Office    20.626151
Name: Same Day, dtype: float64

In [237]:
shipmode_order.sum() / sum(shipmode_order.sum()) * 100

Segment
Consumer       51.941165
Corporate      30.218131
Home Office    17.840704
dtype: float64

>Segmen Consumer mendominasi untuk order pada semua jenis ship mode. Hal ini berkorelasi dengan Segmen Consumer mendominasi sebanyak 50% dari total order yang dilakukan.

### Jenis pengiriman barang seperti apakah yang memberikan profit terbesar?

In [238]:
shipmode_profit = retail[['Row ID', 'Ship Mode', 'Profit']].copy()

In [243]:
shipmode_profit.groupby('Ship Mode')['Profit'].sum().sort_values(ascending=False)

Ship Mode
Standard Class    164088.7875
Second Class       57446.6354
First Class        48969.8399
Same Day           15891.7589
Name: Profit, dtype: float64

In [244]:
shipmode_profit.groupby('Ship Mode')['Profit'].mean().sort_values(ascending=False)

Ship Mode
First Class       31.839948
Second Class      29.535545
Same Day          29.266591
Standard Class    27.494770
Name: Profit, dtype: float64

In [245]:
shipmode_profit['Ship Mode'].value_counts()

Standard Class    5968
Second Class      1945
First Class       1538
Same Day           543
Name: Ship Mode, dtype: int64

>Secara keseluruhan Ship Mode Standard Class memberikan profit yang paling tinggi walaupun profit untuk setiap transaksinya terendah jika dibandingkan dengan ship mode yang lain. Hal ini dikarenakan frekuensi order dengan menggunakan Ship Mode jenis ini juga tinggi.

In [251]:
retail.describe(include=['datetime64'])

,Order Date,Ship Date
count,9994,9994
unique,1237,1334
top,2016-09-05 00:00:00,2015-12-16 00:00:00
freq,38,35
first,2014-01-03 00:00:00,2014-01-07 00:00:00
last,2017-12-30 00:00:00,2018-01-05 00:00:00
